In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras import Sequential,layers
import tensorflow_hub as hub

In [15]:
import os
import shutil
import zipfile

zip_file_path = '/content/archive (38).zip'
extracted_file_path = '/content/extracted file'

with zipfile.ZipFile(zip_file_path,'r') as zip_ref:
  zip_ref.extractall(extracted_file_path)

In [16]:
test_fake_dir = '/content/extracted file/test/FAKE'

os.makedirs(os.path.join(test_fake_dir,'class1'),exist_ok = True)

for filename in os.listdir(test_fake_dir):
  file_path = os.path.join(test_fake_dir,filename)
  if os.path.isfile(file_path):
    shutil.move(file_path,os.path.join(test_fake_dir,'class1',filename))

In [17]:
test_real_dir = '/content/extracted file/test/REAL'

os.makedirs(os.path.join(test_real_dir,'class1'),exist_ok = True)

for filename in os.listdir(test_real_dir):
  file_path = os.path.join(test_real_dir,filename)
  if os.path.isfile(file_path):
    shutil.move(file_path,os.path.join(test_real_dir,'class1',filename))

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_real_generator = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    horizontal_flip = True,
    zoom_range = 0.2
)

test_fake_generator = ImageDataGenerator(
    rescale = 1./255
)


test_real_dataset = test_real_generator.flow_from_directory(
    test_real_dir,
    target_size = (150,150),
    batch_size = 32
)


test_fake_dataset = test_fake_generator.flow_from_directory(
    test_fake_dir,
    target_size = (150,150),
    batch_size = 32
)

Found 10000 images belonging to 1 classes.
Found 10000 images belonging to 1 classes.


In [19]:
model = Sequential([
    layers.Conv2D(32,(3,3),activation = 'relu',input_shape = (150,150,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation = 'relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128,(3,3),activation = 'relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(512,activation = 'relu'),
    layers.Dense(1,activation = 'sigmoid')
])


model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.fit(test_real_dataset,epochs = 2,validation_data = test_fake_dataset)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


313/313 ━━━━━━━━━━━━━━━━━━━━ 76s 223ms/step - accuracy: 0.9987 - loss: 0.0137 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/2
313/313 ━━━━━━━━━━━━━━━━━━━━ 69s 217ms/step - accuracy: 1.0000 - loss: 1.4153e-43 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


In [20]:
from tensorflow.keras.applications import DenseNet169

base_model = DenseNet169(include_top = False)
base_model.trainable = False

inputs = layers.Input(shape = (150,150,3))
x = base_model(inputs,training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1,activation = 'sigmoid')(x)

model = tf.keras.Model(inputs = inputs,outputs = x)

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.fit(test_real_dataset,epochs = 1,validation_data = test_fake_dataset)

base_model.trainable = True

model.compile(loss = 'binary_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

model.fit(test_real_dataset,epochs = 1,validation_data = test_fake_dataset)

313/313 ━━━━━━━━━━━━━━━━━━━━ 145s 353ms/step - accuracy: 0.9567 - loss: 0.0956 - val_accuracy: 1.0000 - val_loss: 0.0014
313/313 ━━━━━━━━━━━━━━━━━━━━ 455s 776ms/step - accuracy: 1.0000 - loss: 4.7299e-06 - val_accuracy: 1.0000 - val_loss: 4.4643e-08
